In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Orbital Collision Predictor - Data Exploration\n",
    "\n",
    "This notebook demonstrates how to fetch and explore real satellite TLE data from NORAD/Celestrak."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Setup imports\n",
    "import sys\n",
    "sys.path.append('..')\n",
    "\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import plotly.graph_objects as go\n",
    "import plotly.express as px\n",
    "from datetime import datetime, timedelta\n",
    "\n",
    "from src.data.tle_fetcher import TLEFetcher\n",
    "from src.utils.orbital_mechanics import OrbitalMechanics\n",
    "from src.visualization.orbit_visualizer import OrbitVisualizer"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Fetch Real Satellite Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize TLE fetcher\n",
    "fetcher = TLEFetcher()\n",
    "\n",
    "# Fetch different satellite groups\n",
    "print(\"Fetching satellite data...\")\n",
    "satellites_df = fetcher.fetch_all_sources([\"space_stations\", \"starlink\", \"active_satellites\"])\n",
    "\n",
    "print(f\"\\nTotal satellites fetched: {len(satellites_df)}\")\n",
    "print(f\"Data sources: {satellites_df['source'].value_counts().to_dict()}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Explore Satellite Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Display sample data\n",
    "print(\"Sample satellite data:\")\n",
    "display(satellites_df.head())\n",
    "\n",
    "# Data statistics\n",
    "print(\"\\nData columns:\")\n",
    "print(satellites_df.columns.tolist())\n",
    "\n",
    "print(\"\\nOrbital parameters statistics:\")\n",
    "orbital_params = ['inclination', 'eccentricity', 'mean_motion', 'raan', 'arg_perigee']\n",
    "display(satellites_df[orbital_params].describe())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualize orbital parameters distribution\n",
    "fig = px.scatter(\n",
    "    satellites_df.sample(min(1000, len(satellites_df))),\n",
    "    x='inclination',\n",
    "    y='mean_motion',\n",
    "    color='source',\n",
    "    title='Satellite Distribution: Inclination vs Mean Motion',\n",
    "    labels={'inclination': 'Inclination (degrees)', 'mean_motion': 'Mean Motion (revs/day)'}\n",
    ")\n",
    "fig.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Find International Space Station (ISS)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Find ISS\n",
    "iss_data = satellites_df[satellites_df['name'].str.contains('ISS', case=False)]\n",
    "\n",
    "if not iss_data.empty:\n",
    "    iss = iss_data.iloc[0]\n",
    "    print(f\"Found: {iss['name']}\")\n",
    "    print(f\"NORAD ID: {iss['norad_id']}\")\n",
    "    print(f\"Inclination: {iss['inclination']:.2f}°\")\n",
    "    print(f\"Mean Motion: {iss['mean_motion']:.2f} revs/day\")\n",
    "    print(f\"Approximate altitude: {550 + (15.5 - iss['mean_motion']) * 50:.0f} km\")\n",
    "else:\n",
    "    print(\"ISS not found in data\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Propagate and Visualize ISS Orbit"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if not iss_data.empty:\n",
    "    # Initialize orbital mechanics\n",
    "    om = OrbitalMechanics()\n",
    "    viz = OrbitVisualizer()\n",
    "    \n",
    "    # Create satellite record from TLE\n",
    "    satrec = om.tle_to_satrec(iss['tle_line1'], iss['tle_line2'])\n",
    "    \n",
    "    # Propagate orbit for next 2 hours\n",
    "    start_time = datetime.utcnow()\n",
    "    orbit_data = om.propagate_orbit(satrec, start_time, duration_hours=2.0)\n",
    "    \n",
    "    print(f\"Propagated {len(orbit_data['times'])} positions\")\n",
    "    print(f\"Current altitude: {orbit_data['altitudes'][0]:.1f} km\")\n",
    "    \n",
    "    # Create 3D visualization\n",
    "    fig = go.Figure()\n",
    "    \n",
    "    # Add Earth\n",
    "    fig.add_trace(viz.create_earth_sphere())\n",
    "    \n",
    "    # Add ISS orbit\n",
    "    iss_traces = viz.plot_orbit_3d(orbit_data, iss['name'])\n",
    "    for trace in iss_traces:\n",
    "        fig.add_trace(trace)\n",
    "    \n",
    "    fig.update_layout(\n",
    "        title='ISS Orbit Visualization (Next 2 Hours)',\n",
    "        scene=dict(\n",
    "            xaxis_title='X (km)',\n",
    "            yaxis_title='Y (km)',\n",
    "            zaxis_title='Z (km)',\n",
    "            aspectmode='data'\n",
    "        ),\n",
    "        width=900,\n",
    "        height=700\n",
    "    )\n",
    "    \n",
    "    fig.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Analyze Starlink Constellation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Filter Starlink satellites\n",
    "starlink_sats = satellites_df[satellites_df['source'] == 'starlink']\n",
    "print(f\"Found {len(starlink_sats)} Starlink satellites\")\n",
    "\n",
    "# Analyze orbital shells\n",
    "if len(starlink_sats) > 0:\n",
    "    # Approximate altitude from mean motion\n",
    "    starlink_sats['approx_altitude'] = 550 + (15.5 - starlink_sats['mean_motion']) * 50\n",
    "    \n",
    "    # Plot altitude distribution\n",
    "    fig = px.histogram(\n",
    "        starlink_sats,\n",
    "        x='approx_altitude',\n",
    "        nbins=50,\n",
    "        title='Starlink Satellites Altitude Distribution',\n",
    "        labels={'approx_altitude': 'Approximate Altitude (km)', 'count': 'Number of Satellites'}\n",
    "    )\n",
    "    fig.show()\n",
    "    \n",
    "    # Inclination groups\n",
    "    fig = px.histogram(\n",
    "        starlink_sats,\n",
    "        x='inclination',\n",
    "        nbins=30,\n",
    "        title='Starlink Satellites Inclination Distribution',\n",
    "        labels={'inclination': 'Inclination (degrees)', 'count': 'Number of Satellites'}\n",
    "    )\n",
    "    fig.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Identify Potential Close Approaches"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Find satellites with similar orbital parameters (potential for close approaches)\n",
    "# This is a simplified check - actual collision detection is more complex\n",
    "\n",
    "# Focus on LEO satellites\n",
    "leo_sats = satellites_df[satellites_df['mean_motion'] > 14.0].copy()\n",
    "leo_sats['approx_altitude'] = 550 + (15.5 - leo_sats['mean_motion']) * 50\n",
    "\n",
    "# Find satellites at similar altitudes\n",
    "altitude_threshold = 50  # km\n",
    "target_altitude = 550  # km (busy altitude)\n",
    "\n",
    "similar_altitude_sats = leo_sats[\n",
    "    (leo_sats['approx_altitude'] > target_altitude - altitude_threshold) &\n",
    "    (leo_sats['approx_altitude'] < target_altitude + altitude_threshold)\n",
    "]\n",
    "\n",
    "print(f\"Found {len(similar_altitude_sats)} satellites near {target_altitude} km altitude\")\n",
    "\n",
    "# Group by inclination to find potential crossing orbits\n",
    "if len(similar_altitude_sats) > 0:\n",
    "    inclination_groups = similar_altitude_sats.groupby(pd.cut(similar_altitude_sats['inclination'], bins=10))\n",
    "    print(\"\\nSatellites by inclination group:\")\n",
    "    for inc_range, group in inclination_groups:\n",
    "        if len(group) > 0:\n",
    "            print(f\"  {inc_range}: {len(group)} satellites\")"